In [42]:
'''
Tasks:

-- Date preprocessing -- 
    converting dates between languages, and then structure
    --> alternatively, we can duplicate the set, change weird dates to none
    
--feature creation--
    date by year
    looking at if they are posting weekday/weekend might suggest that they are working age
    language (foreign in general or specifics)
    number of posts per user.id
    
-- text --
    remove the stopwords
        NLTK
    remove the punctuation
    stemming the words
    SVD (singular value decomposition) --> will show what is most important by 
        taking the matrix of term counts (needs to be created), but returns importance values ~ variability with each word
    TF-IDF: could filter down to top 500-1000 words (we can probably just implement using a package) 
        if not...
        calculate the frequency of word in every document
        inverse log 
        
-- train model: multiple linear regression --
    glm net --> just make sure not to do log, but can still use this for normal parametric
        (generalization of ridge and lasso regression)
    
    CV procedure to judge accuracy of predictor based on words we keep or parameters of the linear model

    
general notes: what is a document?

    post?
        -this would let us use the distribution of predictions for a user to do a second layer to the model
        -potentially average/take median of different predictions for individual documents
        
        -might want to consider the number of unique pairings between user.id and post.id
        
    single user?
    
    bigrams or trigrams instead of words (two or three words in order can be treated as words)
        n words in sentence --> n-1 bigrams --> blows up dimensionality, but could give more information because context
        comes into play (may or may not be feasible depending on our corpus)
        
        
Note from Sean: We may want to use the "gensim" package, which is Google's pre-trained word2vec model, which I think could be really useful.

'''
;

''

In [57]:
import os
import numpy as np
import pandas as pd
import sklearn as slr
import nltk

In [6]:
# Set wd
#os.chdir("C://Users/alxgr//Documents//UVA//DSI//Fall 2018//SYS//Kaggle Competition 3//sys6018-competition-blogger-characteristics")
os.chdir("/Users/SM/DSI/classes/fall2018/SYS6018/kaggle/sys6018-competition-blogger-characteristics/")

In [10]:
# read in data
train = pd.read_csv(os.path.join("data", "input", "train.csv"))
test = pd.read_csv(os.path.join("data", "input", "test.csv"))

In [11]:
# start data exploration
train.head()

,post.id,user.id,gender,topic,sign,date,text,age
0,1,11869,male,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,...",15
1,2,11869,male,Student,Leo,"13,May,2004",These are the team members: Drewe...,15
2,3,11869,male,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...,15
3,4,11869,male,Student,Leo,"12,May,2004",testing!!! testing!!!,15
4,5,16332,male,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...,33


In [12]:
pd.set_option('display.max_rows', 5000)
train.groupby(['user.id', 'post.id']).size()

user.id  post.id
1        592732     1
         592733     1
         592734     1
         592735     1
         592736     1
         592737     1
         592738     1
         592739     1
         592740     1
         592741     1
         592742     1
         592743     1
         592744     1
         592745     1
         592746     1
         592747     1
         592748     1
         592749     1
         592750     1
         592751     1
         592752     1
         592753     1
         592754     1
         592755     1
         592756     1
         592757     1
         592758     1
         592759     1
         592760     1
         592761     1
         592762     1
3        282649     1
         282650     1
         282651     1
         282652     1
         282653     1
         282654     1
         282655     1
         282656     1
4        563519     1
5        263002     1
         263003     1
         263004     1
         263005     1
         263006

In [13]:
train.shape

(442961, 8)

In [14]:
train.describe()

,post.id,user.id,age
count,442961.000000,442961.000000,442961.000000
mean,340076.005231,9655.198203,23.540594
std,196532.771521,5512.481177,7.603127
min,1.000000,1.000000,13.000000
25%,168419.000000,4760.000000,17.000000
50%,341752.000000,9630.000000,24.000000
75%,510601.000000,14278.000000,26.000000
max,681161.000000,19319.000000,48.000000


In [15]:
train.isnull().sum()
test.isnull().sum()

post.id    0
user.id    0
gender     0
topic      0
sign       0
date       0
text       0
dtype: int64

In [16]:
train.topic.value_counts()

indUnk                     159419
Student                    102746
Technology                  28346
Arts                        24313
Education                   18563
Communications-Media        13195
Non-Profit                   9036
Internet                     8865
Engineering                  7139
Publishing                   5614
Law                          5502
Science                      4876
Consulting                   4540
Government                   3946
Religion                     3466
BusinessServices             3454
Marketing                    3130
Fashion                      2755
Chemicals                    2680
Museums-Libraries            2636
Sports-Recreation            2523
Advertising                  2502
Telecommunications           2464
HumanResources               2274
Banking                      2199
Accounting                   2185
Military                     1795
RealEstate                   1672
Transportation               1517
Manufacturing 

In [17]:
train.sign.value_counts()

Taurus         42051
Libra          41088
Virgo          40923
Scorpio        38548
Cancer         38104
Leo            36809
Aries          36714
Aquarius       35634
Sagittarius    34692
Pisces         32846
Gemini         32831
Capricorn      32721
Name: sign, dtype: int64

In [18]:
# join data sets
test['age'] = None
train['set'] = "train"
test['set'] = "test"

In [19]:
test.head()

,post.id,user.id,gender,topic,sign,date,text,age,set
0,96,4876,female,indUnk,Capricorn,"08,August,2004","urlLink im new to this, ...",None,test
1,97,4876,female,indUnk,Capricorn,"08,August,2004",Election time has rolled aro...,None,test
2,108,12227,female,Student,Aries,"31,July,2004","Met Lim this morning, then we...",None,test
3,109,12227,female,Student,Aries,"30,July,2004",Such a boring day. I have no ...,None,test
4,110,12227,female,Student,Aries,"29,July,2004","Nuts. As usual, Theo and I pl...",None,test


In [65]:
alldata = pd.concat([train, test])

In [66]:
alldata.isnull().sum()

post.id         0
user.id         0
gender          0
topic           0
sign            0
date            0
text            0
age        238323
set             0
dtype: int64

In [68]:
# convert to datetime, leaving non-valid/non-English dates null
alldata['date_date'] = pd.to_datetime(alldata.date, errors='coerce')

In [79]:
# Remove rows where date cannot be parsed - these are non-English and so unless translated
# into English, cannot be used in a text model with this corpus
alldata = alldata[alldata['date_date'].notnull()]
alldata.date = alldata.date_date
alldata = alldata.drop(['date_date'], axis=1)

In [107]:
# adjusting age to age at date of posting by subtracting post date from max(post dates)
# This may be a bad idea if ages are already age at time of posting. I think that is actually the case upon inspection.
alldata['adj_age'] = alldata.age - (2006 - alldata['date'].dt.year)
alldata.adj_age = alldata.adj_age.astype(float)

235046

In [109]:
alldata.head()

,post.id,user.id,gender,topic,sign,date,text,age,set,year,adj_age
0,1,11869,male,Student,Leo,2004-05-14,"Info has been found (+/- 100 pages,...",15,train,2004,13.0
1,2,11869,male,Student,Leo,2004-05-13,These are the team members: Drewe...,15,train,2004,13.0
2,3,11869,male,Student,Leo,2004-05-12,In het kader van kernfusie op aarde...,15,train,2004,13.0
3,4,11869,male,Student,Leo,2004-05-12,testing!!! testing!!!,15,train,2004,13.0
4,5,16332,male,InvestmentBanking,Aquarius,2004-06-11,Thanks to Yahoo!'s Toolbar I can ...,33,train,2004,31.0


In [110]:
alldata.describe()

,post.id,user.id,year,adj_age
count,673538.000000,673538.00000,673538.000000,438492.000000
mean,340564.101435,9588.18959,2003.750703,21.318742
std,196611.977137,5682.78550,0.567003,7.604802
min,1.000000,1.00000,1999.000000,6.000000
25%,170738.250000,4379.00000,2004.000000,14.000000
50%,340700.500000,9726.00000,2004.000000,22.000000
75%,510687.750000,14498.00000,2004.000000,24.000000
max,681284.000000,19320.00000,2006.000000,46.000000
